In [1]:
from optimum.onnxruntime import ORTStableDiffusionXLPipeline
# from optimum.onnxruntime im
import time
start = time.time()
model_id = "stabilityai/stable-diffusion-xl-base-1.0"
base = ORTStableDiffusionXLPipeline.from_pretrained(
    model_id, 
    # provider="CUDAExecutionProvider",
    device="cuda:1"
)
end = time.time()
print("Time taken to load the model: ", end-start)

/home/tiennv/.conda/envs/inpaint/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 23 files: 100%|██████████| 23/23 [00:00<00:00, 42799.02it/s]


Time taken to load the model:  160.9673788547516


: 

In [4]:
unet = base.unet
del base

In [ ]:
# sample shape: (2, 4, 128, 128)
# timestep shape: (1,)
# encoder_hidden_states shape: (2, 77, 2048)
# text_embeds shape: (2, 1280)
# time_ids shape: (2, 6)
# --------------------
# sample type: float32
# timestep type: int64
# encoder_hidden_states type: float32
# text_embeds type: float32
# time_ids type: float32

In [12]:
import torch
noisy_image = torch.randn(1, 4, 64, 64)
# time_step = torch.randn(1)
time_step = [5]
prompt_embed = torch.randn(1, 77, 2048)
pooled_output = torch.randn(1, 1280)
add_time_ids = torch.randn(1, 6)
unet_added_conditions = {"time_ids": add_time_ids}
unet_added_conditions.update({"text_embed": pooled_output})

input_unet = {
    "sample": noisy_image,
    "timestep": time_step,
    "encoder_hidden_states": prompt_embed,
    "text_embeds": pooled_output,
    "time_ids": add_time_ids
}

# convert tensor to numpy array
for key, value in input_unet.items():
    try:
        input_unet[key] = value.cpu().numpy()
    except:
        pass
# convert timestep to int64
# input_unet['timestep'] = input_unet['timestep'].astype('int64')
import time
start = time.time()
output = unet(
    sample=input_unet['sample'],
    timestep=input_unet['timestep'],
    encoder_hidden_states=input_unet['encoder_hidden_states'],
    text_embeds=input_unet['text_embeds'],
    time_ids=input_unet['time_ids'],
)
end = time.time()
print(f"Time: {end-start}")

Time: 2.7880680561065674


In [15]:
output 

[array([[[[-9.31797266e-01, -1.54932383e-02, -9.62544829e-02, ...,
            5.61438322e-01, -4.36396062e-01, -3.93198609e-01],
          [-2.98084974e+00,  1.03432322e+00,  5.23259223e-01, ...,
           -4.58927780e-01, -5.10520220e-01,  3.45975578e-01],
          [ 2.17444682e+00,  1.77213252e-01, -5.20087183e-01, ...,
           -7.84323588e-02,  1.91367960e+00,  3.62004265e-02],
          ...,
          [ 4.86770570e-01, -2.89057910e-01, -1.61813796e-01, ...,
            1.01563859e+00,  1.23358659e-01,  3.90638560e-01],
          [-2.07842201e-01,  9.20455083e-02,  1.22159779e-01, ...,
           -1.98932600e+00,  2.65264392e-01, -9.83081162e-01],
          [ 6.64047599e-01,  3.42296183e-01, -2.76596332e+00, ...,
            2.10247850e+00, -1.82114053e+00,  1.83230698e-01]],
 
         [[-1.56654000e-01,  2.61764669e+00, -8.10156703e-01, ...,
           -4.75424618e-01, -1.17601347e+00,  1.57504749e+00],
          [ 1.11268401e+00,  1.33206010e+00, -7.63212264e-01, ...,
     